In [2]:

import pandas as pd

# Define file paths
file_2022 = '../data/2022-10-01-PB NDIS Support Catalogue 2022-23 .xlsx'
file_2023_v1 = '../data/2023-05-05-NDIS Support Catalogue 2022-23 v1.1_0.xlsx'
file_2023_v2 = '../data/PB NDIS Support Catalogue 2023-24  - Current Support Items.csv'

# Load the spreadsheets into dataframes
df_2022 = pd.read_excel(file_2022)
df_2023_v1 = pd.read_excel(file_2023_v1)
df_2023_v2 = pd.read_csv(file_2023_v2)

# Show the first few rows and columns of each dataframe to understand their structure
df_2022.head(), df_2023_v1.head(), df_2023_v2.head()

ImportError: Missing optional dependency 'openpyxl'.  Use pip or conda to install openpyxl.

In [ ]:
# Normalize columns
def normalize_columns(df):
    return df.rename(columns=lambda x: x.lower().replace(" ", "_").replace("-", "_"))

df_2022 = normalize_columns(df_2022)
df_2023_v1 = normalize_columns(df_2023_v1)
df_2023_v2 = normalize_columns(df_2023_v2)

# Display normalized column names for each dataframe
df_2022.columns, df_2023_v1.columns, df_2023_v2.columns


In [ ]:
# Identify item codes present in each dataframe
item_codes_2022 = set(df_2022['support_item_number'])
item_codes_2023_v1 = set(df_2023_v1['support_item_number'])
item_codes_2023_v2 = set(df_2023_v2['support_item_number'])

# Find new and removed item codes
new_codes_2023_v1 = item_codes_2023_v1 - item_codes_2022
new_codes_2023_v2 = item_codes_2023_v2 - item_codes_2023_v1
removed_codes_2023_v1 = item_codes_2022 - item_codes_2023_v1
removed_codes_2023_v2 = item_codes_2023_v1 - item_codes_2023_v2

In [ ]:
# Define the output columns and create empty dataframes to store the differences
output_columns = ['item_code', 'status', 'feature_changed', 'previous_value', 'new_value', 'date_new_value_effective', 'filename_new_value', 'filename_old_value']
df_diff_2022_2023_v1 = pd.DataFrame(columns=output_columns)
df_diff_2023_v1_2023_v2 = pd.DataFrame(columns=output_columns)

# Find common item codes for comparison
common_codes_2022_2023_v1 = item_codes_2022.intersection(item_codes_2023_v1)
common_codes_2023_v1_2023_v2 = item_codes_2023_v1.intersection(item_codes_2023_v2)


# Modify the function to populate the new columns
def check_changes(df1, df2, common_codes, date_new_value_effective, filename_new_value, filename_old_value, df_diff):
    common_columns = set(df1.columns).intersection(set(df2.columns))
    for code in common_codes:
        row1 = df1[df1['support_item_number'] == code].iloc[0]
        row2 = df2[df2['support_item_number'] == code].iloc[0]
        for col in common_columns:
            if row1[col] != row2[col]:
                df_diff = df_diff.append({
                    'item_code': code,
                    'status': 'Changed',
                    'feature_changed': col,
                    'previous_value': row1[col],
                    'new_value': row2[col],
                    'date_new_value_effective': date_new_value_effective,
                    'filename_new_value': filename_new_value,
                    'filename_old_value': filename_old_value
                }, ignore_index=True)
    return df_diff

# Add new and removed codes to the dataframes
def add_new_removed_codes(new_codes, removed_codes, date_new_value_effective, filename_new_value, filename_old_value, df_diff):
    for code in new_codes:
        df_diff = df_diff.append({
            'item_code': code,
            'status': 'New',
            'feature_changed': 'N/A',
            'previous_value': 'N/A',
            'new_value': 'N/A',
            'date_new_value_effective': date_new_value_effective,
            'filename_new_value': filename_new_value,
            'filename_old_value': filename_old_value
        }, ignore_index=True)
    for code in removed_codes:
        df_diff = df_diff.append({
            'item_code': code,
            'status': 'Removed',
            'feature_changed': 'N/A',
            'previous_value': 'N/A',
            'new_value': 'N/A',
            'date_new_value_effective': date_new_value_effective,
            'filename_new_value': filename_new_value,
            'filename_old_value': filename_old_value
        }, ignore_index=True)
    return df_diff

# Add new and removed codes for each comparison
df_diff_2022_2023_v1 = add_new_removed_codes(new_codes_2023_v1, removed_codes_2023_v1, '2023-05-05', '2023-05-05-NDIS Support Catalogue 2022-23 v1.1_0.xlsx', '2022-10-01-PB NDIS Support Catalogue 2022-23 .xlsx', df_diff_2022_2023_v1)
df_diff_2023_v1_2023_v2 = add_new_removed_codes(new_codes_2023_v2, removed_codes_2023_v2, '2023-07-01', 'PB NDIS Support Catalogue 2023-24  - Current Support Items.csv', '2023-05-05-NDIS Support Catalogue 2022-23 v1.1_0.xlsx', df_diff_2023_v1_2023_v2)

# Run the check_changes function for each comparison
df_diff_2022_2023_v1 = check_changes(df_2022, df_2023_v1, common_codes_2022_2023_v1, '2023-05-05', '2023-05-05-NDIS Support Catalogue 2022-23 v1.1_0.xlsx', '2022-10-01-PB NDIS Support Catalogue 2022-23 .xlsx', df_diff_2022_2023_v1)
df_diff_2023_v1_2023_v2 = check_changes(df_2023_v1, df_2023_v2, common_codes_2023_v1_2023_v2, '2023-07-01', 'PB NDIS Support Catalogue 2023-24  - Current Support Items.csv', '2023-05-05-NDIS Support Catalogue 2022-23 v1.1_0.xlsx', df_diff_2023_v1_2023_v2)

# Save the results to CSV files
output_file_path_2022_2023_v1 = '/mnt/data/item_code_differences_2022_2023_v1.csv'
output_file_path_2023_v1_2023_v2 = '/mnt/data/item_code_differences_2023_v1_2023_v2.csv'
df_diff_2022_2023_v1.to_csv(output_file_path_2022_2023_v1, index=False)
df_diff_2023_v1_2023_v2.to_csv(output_file_path_2023_v1_2023_v2, index=False)

output_file_path_2022_2023_v1, output_file_path_2023_v1_2023_v2


In [ ]:
# Modify the function to correctly populate the 'previous_value' and 'new_value' columns for changed items
def check_changes(df1, df2, common_codes, date_new_value_effective, filename_new_value, filename_old_value, df_diff):
    common_columns = set(df1.columns).intersection(set(df2.columns))
    for code in common_codes:
        row1 = df1[df1['support_item_number'] == code].iloc[0]
        row2 = df2[df2['support_item_number'] == code].iloc[0]
        for col in common_columns:
            if pd.notna(row1[col]) and pd.notna(row2[col]) and row1[col] != row2[col]:
                df_diff = df_diff.append({
                    'item_code': code,
                    'status': 'Changed',
                    'feature_changed': col,
                    'previous_value': row1[col],
                    'new_value': row2[col],
                    'date_new_value_effective': date_new_value_effective,
                    'filename_new_value': filename_new_value,
                    'filename_old_value': filename_old_value
                }, ignore_index=True)
    return df_diff

# Clear df_diff to start fresh
df_diff_2022_2023_v1 = pd.DataFrame(columns=output_columns)
df_diff_2023_v1_2023_v2 = pd.DataFrame(columns=output_columns)

# Add new and removed codes for each comparison
df_diff_2022_2023_v1 = add_new_removed_codes(new_codes_2023_v1, removed_codes_2023_v1, '2023-05-05', '2023-05-05-NDIS Support Catalogue 2022-23 v1.1_0.xlsx', '2022-10-01-PB NDIS Support Catalogue 2022-23 .xlsx', df_diff_2022_2023_v1)
df_diff_2023_v1_2023_v2 = add_new_removed_codes(new_codes_2023_v2, removed_codes_2023_v2, '2023-07-01', 'PB NDIS Support Catalogue 2023-24  - Current Support Items.csv', '2023-05-05-NDIS Support Catalogue 2022-23 v1.1_0.xlsx', df_diff_2023_v1_2023_v2)

# Run the check_changes function for each comparison
df_diff_2022_2023_v1 = check_changes(df_2022, df_2023_v1, common_codes_2022_2023_v1, '2023-05-05', '2023-05-05-NDIS Support Catalogue 2022-23 v1.1_0.xlsx', '2022-10-01-PB NDIS Support Catalogue 2022-23 .xlsx', df_diff_2022_2023_v1)
df_diff_2023_v1_2023_v2 = check_changes(df_2023_v1, df_2023_v2, common_codes_2023_v1_2023_v2, '2023-07-01', 'PB NDIS Support Catalogue 2023-24  - Current Support Items.csv', '2023-05-05-NDIS Support Catalogue 2022-23 v1.1_0.xlsx', df_diff_2023_v1_2023_v2)

# Save the results to CSV files
output_file_path_2022_2023_v1 = '/mnt/data/item_code_differences_2022_2023_v1_corrected.csv'
output_file_path_2023_v1_2023_v2 = '/mnt/data/item_code_differences_2023_v1_2023_v2_corrected.csv'
df_diff_2022_2023_v1.to_csv(output_file_path_2022_2023_v1, index=False)
df_diff_2023_v1_2023_v2.to_csv(output_file_path_2023_v1_2023_v2, index=False)

output_file_path_2022_2023_v1, output_file_path_2023_v1_2023_v2
